In [ ]:
# This allows importing Jupyter notebooks as modules
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import JupyterNotebookImporter

In [ ]:
#import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, LSTM
import keras.backend as kb
import keras
from sklearn.metrics import mean_squared_error
import numpy as np
from keras import regularizers, optimizers, initializers
from numpy.random import seed
import AlgoPlotting as plt
#from AlgoPlotting import XYChart

In [ ]:
def shape_it(X):
    return np.expand_dims(X.reshape((-1,1)),2)

In [ ]:
# # Generate data
# data = np.random.randint(-5,5,200)

# n_data = len(data)
# data = np.matrix(data)
# n_train = int(0.8*n_data)

# index_delay = 5
# y_train = shape_it(data[:,:n_train])
# x_train = shape_it(data[:,index_delay:(n_train+index_delay)])
# y_test = shape_it(data[:,n_train:-index_delay])
# x_test = shape_it(data[:,(n_train+index_delay):])

In [ ]:
def generate_data():
    
    #data = np.random.randint(-1,2,200)
    #data = 2*(np.random.rand(200) - 0.5)
    #data = np.cumsum(data)
    #x = data
    #y = [data[ii]+data[ii+1]*data[ii+2] for ii in range(200)]
    
    n = np.random.randint(5,15)
    
    x = []
    for ii in range(n):
        if 1 or np.random.rand() > 0.01 + 0.99 * ii/200:
            x.append(-1)
        else:
            x.append(1)
    
    y = x[1:]
    x = x[0:-1]
        
#     print(x)
#     print(y)
#     print(np.shape(x))
#     print(np.shape(y))
    
    x = shape_it(np.array(x))
    y = shape_it(np.array(y))
    return x, y
    
#generate_data()

In [ ]:
# Plot training data
# training_chart = plt.XYChart(y=[x_train, y_train],
#                              names=['X Train','Y Train'],
#                              title='Training Data'
#                             )

In [ ]:
# Plot testing data
# testing_chart = plt.XYChart(y=[x_test, y_test],
#                             names=['X Test','Y Test'],
#                             title='Testing Data'
#                            )

In [ ]:
class SGDLearningRateTracker(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        opt = self.model.optimizer
        
        # Not sure why but eval'ing 1 by 1 then calculating doesn't crash, but 1 line does
        lr = kb.eval(opt.lr)
        decay = kb.eval(opt.decay)
        iterations = kb.eval(opt.iterations)
        #lr = kb.eval(opt.lr * (1. / (1. + opt.decay * opt.iterations))) # This crashes
        lr = (lr * (1. / (1. + decay * iterations)))
        #print('\nLR: {:.6f}\n'.format(lr))
        logs['lr_calc'] = lr

In [ ]:
class PatternPredictingNetwork:
    def __init__(self, model_dict, x_train, y_train, x_test, y_test):
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        self.model = Sequential()
        
        batch_size = 1
        los = 'mse'
        act = 'linear'
        learning_rate = 0.00001*5
        decay_rate = 0.00001
        momentum = 0.0
        sgd = optimizers.SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
        opt = sgd
        kreg = regularizers.l2(0.0001)
        areg = regularizers.l1(0.0001)
        #neurons = 300 # Works for delay = 5, SimpleRNN
        neurons = 1
        #batch_input_shape=(batch_size, self.x_train.shape[1], self.x_train.shape[2])
        batch_input_shape=(batch_size, 1, 1)
        use_regularization = False

        if use_regularization:
            self.model.add(SimpleRNN(neurons, batch_input_shape=batch_input_shape, stateful=True, return_sequences=True,
                            kernel_regularizer=kreg, activity_regularizer=areg))
            self.model.add(SimpleRNN(neurons, batch_input_shape=batch_input_shape, stateful=True, return_sequences=True,
                            kernel_regularizer=kreg, activity_regularizer=areg))
            self.model.add(SimpleRNN(neurons, batch_input_shape=batch_input_shape, stateful=True,
                            kernel_regularizer=kreg, activity_regularizer=areg))
            self.model.add(Dense(neurons, kernel_regularizer=kreg, activity_regularizer=areg, activation=act))
            self.model.add(Dense(1, kernel_regularizer=kreg, activity_regularizer=areg))
            self.model.compile(loss=los, optimizer=opt)
        else:
            #self.model.add(SimpleRNN(neurons, batch_input_shape=batch_input_shape, stateful=True, return_sequences=True))
            #self.model.add(SimpleRNN(neurons, batch_input_shape=batch_input_shape, stateful=True, return_sequences=True))
            self.model.add(SimpleRNN(neurons, activation=act, batch_input_shape=batch_input_shape, stateful=True, use_bias=False))
            #self.model.add(Dense(neurons, activation=act))
            #self.model.add(Dense(1))
            self.model.compile(loss=los, optimizer=opt)
        
    def train_and_monitor(self, epochs=100, epoch_update_interval=10, losses_trim=0, losses_window=None):
        predictions = []
        losses = []
        learning_rates = []
        batch_size = 1

        predictions_chart = plt.XYChart(title='Predictions', x_label='Index', y_label='Y', names=['Prediction', 'Test Data'])
        losses_chart = plt.XYChart(title='Losses', x_label='Epochs', y_label='Loss')
        dlosses_chart = plt.XYChart(title='Diff(Losses)', x_label='Epochs', y_label='Diff(Loss)')
        lr_chart = plt.XYChart(title='Learning Rate', x_label='Epochs', y_label='Learning Rate')
        
        once = True
        
        for epoch in range(epochs):
            self.x_train, self.y_train = generate_data()
            #self.x_test, self.y_test = generate_data()
            self.x_test = self.x_train
            self.y_test = self.y_train
            
            if once:
                y_bad = np.reshape(self.y_train,(-1,))
                print(y_bad)
                print(y_bad.shape)
                once = False
            #self.model.predict(shape_it(np.array(-1)), batch_size=batch_size)
            history = self.model.fit(self.x_train, np.reshape(self.y_train,(-1,)), epochs=1,
                                     batch_size=batch_size, verbose=0, shuffle=False, callbacks=[SGDLearningRateTracker()])
#             history = self.model.fit(self.x_train, self.y_train, epochs=1,
#                                      batch_size=batch_size, verbose=0, shuffle=False, callbacks=[SGDLearningRateTracker()])
            loss = history.history['loss'][0]
            lr = history.history['lr_calc'][0]
            losses.append(loss)
            learning_rates.append(lr)

            # Update the plots
            if not epoch % epoch_update_interval:
                self.model.reset_states()
                predictions = []
                for ii in range(len(self.x_test)):
                    # make one-step forecast
                    X = self.x_test[ii]
                    X = X.reshape(1, 1, 1)
                    y_pred = self.model.predict(X, batch_size=batch_size)[0,0]

                    # store forecast
                    predictions.append(y_pred)
                    expected = self.y_test[ii]

                # Push off the starting indices once the losses is long enough
                if losses_window is not None:
                    if len(losses) <= losses_window:
                            y_losses = losses
                            x_losses = np.arange(len(y_losses))
                    else:
                        start = len(losses) - losses_window
                        y_losses = losses[start:]
                        x_losses = np.arange(start, len(losses))
                else:
                    if len(losses) <= losses_trim:
                        y_losses = losses
                        x_losses = np.arange(len(y_losses))
                    elif len(losses) > losses_trim and len(losses) < 2*losses_trim:
                        start = len(losses) - losses_trim
                        y_losses = losses[start:]
                        x_losses = np.arange(start, len(losses))
                    else:
                        y_losses = losses[losses_trim:]
                        x_losses = np.arange(losses_trim, len(losses))

                predictions_chart.update(y=[predictions, self.y_test])
                losses_chart.update(x=x_losses, y=y_losses)
                dlosses_chart.update(x=x_losses[:-1], y=np.diff(y_losses))
                lr_chart.update(y=learning_rates)
            self.model.reset_states()
    
    # Forecast with updates each step
    def one_step_forecast(self):
        batch_size = 1
        # One step forecast on testing data
        self.model.reset_states()
        self.model.predict(self.x_train, batch_size=batch_size)
        predictions = []
        for ii in range(len(self.x_test)):
            # make one-step forecast
            X = self.x_test[ii]
            X = X.reshape(1, 1, 1)
            yhat = self.model.predict(X, batch_size=batch_size)[0,0]

            # store forecast
            predictions.append(yhat)
            expected = self.y_test[ii]
            print('Index=%d, Predicted=%f, Expected=%f' % (ii, yhat, expected))

        # report performance
        rmse = np.sqrt(mean_squared_error(self.y_test.reshape(len(self.y_test)), predictions))
        print('Test RMSE: %.3f' % rmse)
        xy_chart = plt.XYChart(y=[predictions, self.y_test], names=['Prediction', 'Test Data'], title='One Step Prediction')
        
    def dynamic_forecast(self):
        batch_size = 1
        # Dynamic forecast on test data
        self.model.reset_states()
        self.model.predict(self.x_train, batch_size=batch_size)

        dynpredictions = list()
        dyhat = self.x_test[0]


        for ii in range(len(self.x_test)):
            # make one-step forecast
            dyhat = yhat.reshape(1, 1, 1)
            dyhat = model.predict(dyhat, batch_size=batch_size)[0,0]

            # store forecast
            dynpredictions.append(dyhat)
            expected = self.y_test[ii]
            print('Index=%d, Predicted Dynamically=%f, Expected=%f' % (ii, dyhat, expected))


        drmse = np.sqrt(mean_squared_error(Y_test.reshape(len(self.y_test)), dynpredictions))
        print('Test Dynamic RMSE: %.3f' % drmse)
        xy_chart = plt.XYChart(y=[dynpredictions, self.y_test], names=['Dynamic Prediction', 'Test Data'], title='Dynamic Prediction')
        

In [ ]:
# model_dict = {}
# epochs = 1000
# epoch_update_interval = 10
# #pattern_predicting_network = PatternPredictingNetwork(model_dict, x_train, y_train, x_test, y_test)
# pattern_predicting_network = PatternPredictingNetwork(model_dict, np.array(0), np.array(0), np.array(0), np.array(0))
# pattern_predicting_network.model.layers[0].set_weights(
#     #[np.array([[1.048831]]), np.array([[-0.05480201]])]
#     [np.array([[0.95]]), np.array([[0]])]
# )
# pattern_predicting_network.model.reset_states()
# pattern_predicting_network.model.predict(shape_it(np.array(-1)), batch_size=1)
pattern_predicting_network.train_and_monitor(epochs=epochs, epoch_update_interval=epoch_update_interval, losses_window=100)

In [ ]:
for layer in pattern_predicting_network.model.layers:
    print(layer.get_weights())

In [ ]:
pattern_predicting_network.one_step_forecast()

In [ ]:
weights = pattern_predicting_network.model.layers[0].get_weights()
weights = [weights[0][0][0], weights[1][0][0], weights[2][0]]

print(weights)
#(-0.75-weights[2])*weights[1]+(-1)*weights[0]+weights[2]

(-1)*(weights[0]) + weights[2] + (-0.75)*(weights[1])